In [ ]:
#  In this tutorial, several optimization problems are solved 
#  using the ModelingToolkit.jl and GalacticOptim.jl packages in Julia. 

#  The function to be optimized (minimized) is the Rosenbrock function:

#       (a - x)^2 + b * (y - x^2)^2

#  where x and y and variables and a and b are constant parameters.
#  The above function is a common test function for optimization methods. 
#  By inspection, it can be see that the minimum is at x = a, y = a^2 
#  and the function value is zero at this location. Despite having an
#  obvious solution, this problem is still a useful numerical test case
#  the problem is difficult to solve numerically if b = 100 or is greater.

#  The packages ModelingToolkit.jl, GlacticOptim.jl, and Optim.jl
#  need to be installed for this tutorial.

#  Date:  9/12/2021

#  Author:  Doug Frey, UMBC

#  Julia version 1.6.1 was used to create this tutorial

In [ ]:
#  The following package versions were used 
#  for this tutorial:

#
#      Status `C:\Users\Douglas Frey\environment_v161_var1\Project.toml`
#   [a75be94c] GalacticOptim v2.0.3
#   [961ee093] ModelingToolkit v6.4.9
#   [429524aa] Optim v1.4.1
#   [1dea7af3] OrdinaryDiffEq v5.55.1
#   [91a5bcdd] Plots v1.21.3

#  Package versions can be checked by running the following commands.

#  First import Pkg:

       import Pkg

#  The command below is only needed if Julia 1.6.1 is being used in
#  a Jupyter notebook due to a bug in Julia 1.6.1. This command can
#  be deleted otherwise.

       Pkg.DEFAULT_IO[] = stdout

#  Write the status to the output:

       Pkg.status()

In [3]:
#  Make available for use the needed packages:

using ModelingToolkit, GalacticOptim, Optim

In [4]:
#  Example problem 1 

#  Minimize the Rosenbrock function
#  with no constraints

#   Newton Trust Region method is used

#  Define the variables and parameters.

@variables x y 
@parameters a b

#  Define the objective function

objective = (a - x)^2 + b * (y - x^2)^2

#  Create an OptmizationSystem object and name it

@named  sys = OptimizationSystem(objective, [x,y], [a,b])

#  Set initial guess.  Note that on purpose a very bad 
#  initial guess will be used.

u0 = [ x => -1.0, y => -2.0]

#  Set parameters, Note the => operator which creates a
#  pair object.

params = [ a => 1.0, b => 100.0]

#  Define the problem to solve.  Note that the Newton Trust
#  Region method uses both the gradient and  Hessian matrix.
#  These will be determined by automatic differentiation (not
#  finite differences) by specifying grad=true, hess=true below.

prob = OptimizationProblem(sys,u0, params, grad=true, hess=true)

#  Create a callback function to monitor the progress to convergence

callback = function (p,l)
    println("Objective function value =  $l")
    return false
end

#  Use the Newton Trust Region method. Desired error is 10^-12. A maximum
#  of 100 iterations will be performed with objective function value 
#  reported every 5 iterations.  Maximum time limit is 100 seconds:

result = solve(prob, NewtonTrustRegion(), x_abstol = 1e-12, 
                   f_abstol = 1e-12, g_abstol = 1e-12, cb = callback, 
                    show_every = 5, time_limit = 100 , iterations=100)

println(" ")
println("Final value of objective function = ", result.minimum)
println(" ")
println("Final value for x = ", result.minimizer[1])
println(" ")
println("Final value for y = ", result.minimizer[2])


Objective function value =  904.0
Objective function value =  2.4036459427675534
Objective function value =  0.8702016582178861
Objective function value =  0.1175275677662976
Objective function value =  0.00012257826985173384
 
Final value of objective function = 0.0
 
Final value for x = 1.0
 
Final value for y = 1.0


In [ ]:
#  Now examine some results symbolically
#  to show symbolics ability of ModelingToolkit.jl

#  First show the gradient

Symbolics.gradient(objective,[x,y])

In [ ]:
#  Next show the Hessian matrix symbolically

Symbolics.hessian(objective,[x,y])

In [ ]:
#  Next show derivaitve of the objective_function
#  with respect to x symbolically

Dx = Differential(x)

expand_derivatives(Dx(objective))

In [ ]:
#  Example problem 2 

#  Minimize the Rosenbrock function
#  with no constraints

#   The Broyden, Fletcher, Golfarb, Shanno (BFGS) 
#   Quasi-Newton method will be used

#  Define the variables and parameters.

@variables x y 
@parameters a b

#  Define the objective function

objective = (a - x)^2 + b * (y - x^2)^2

#  Create an OptmizationSystem object and name it

@named  sys = OptimizationSystem(objective, [x,y], [a,b])

#  Set initial guess.  Note that on purpose a very bad 
#  initial guess will be used.

u0 = [ x => -1.0, y=>-2.0]

#  Set parameters

params = [ a => 1.0, b => 100.0]

#  Define the problem to solve.  Note that the BFGS method 
#  uses only the gradient (and not the Hessian matrix)
#  since the Hessian is approximated internally by the method.

prob = OptimizationProblem(sys,u0, params, grad=true, hess=false)

#  Create a callback function to monitor the progress to convergence

callback = function (p,l)
    println("Objective function value =  $l")
    return false
end

#  Use the BFGS method. Desired error is 10^-12. A maximum
#  of 100 iterations will be performed with objective function value 
#  reported every 5 iterations.  Maximum time limit is 100 seconds:

result = solve(prob, BFGS(), x_abstol = 1e-12, 
                   f_abstol = 1e-12, g_abstol = 1e-12, cb = callback, 
                    show_every = 5, time_limit = 100 , iterations=100)

println(" ")
println("Final value of objective function = ", result.minimum)
println(" ")
println("Final value for x = ", result.minimizer[1])
println(" ")
println("Final value for y = ", result.minimizer[2])


In [ ]:
#  Example problem 3 

#  Minimize the Rosenbrock function
#  with no constraints

#  The Nelder-Mead method will be used. Note this method
#  does not use either the gradient or Hessian and only
#  uses function evaluations.

#  Define the variables and parameters.

@variables x y 
@parameters a b

#  Remainder of statements are similar to the first two examples
#  so most comments are eliminated below.

objective = (a - x)^2 + b * (y - x^2)^2

@named  sys = OptimizationSystem(objective, [x,y], [a,b])

u0 = [ x => -1.0, y=>-2.0]

params = [ a => 1.0, b => 100.0]

#  The Nelder-Mead method does not use either the gradient or
#  Hessian matrix.  Only function evaluations are used.

prob = OptimizationProblem(sys,u0, params, grad=false, hess=false)

callback = function (p,l)
    println("Objective function value =  $l")
    return false
end

#  The Nelder Mead method will be used.  Since this method is less
#  efficient per iteration, more iterations are needed than before.

result = solve(prob, NelderMead(),x_abstol = 1e-12, 
                   f_abstol = 1e-12, g_abstol = 1e-12, cb = callback, 
                    show_every = 20, time_limit = 100 , iterations=200)

println(" ")
println("Final value of objective function = ", result.minimum)
println(" ")
println("Final value for x = ", result.minimizer[1])
println(" ")
println("Final value for y = ", result.minimizer[2])


In [ ]:
#  Example problem 4

#  Minimize the Rosenbrock function using
#  Particle Swarm method so upper and lower 
#  limits can be used on the search variables.

#  Note that the Particle Swarm method
#  does not use either the gradient or Hessian and only
#  uses function evaluations. The method permits upper and
#  lower limits on the search variables. The method is 
#  loosely based on the behavior of flocking birds trying to
#  find food.

#  The Particle Swarm method is a global optimization method 
#  and not a local optimization method like the other options 
#  shown above.

#  Define the variables and parameters.

@variables x y 
@parameters a b

#  Remainder of statements are similar to the first two examples
#  so most comments are eliminated below.

objective = (a - x)^2 + b * (y - x^2)^2

@named  sys = OptimizationSystem(objective, [x,y], [a,b])

u0 = [ x => -1.0, y=>-1.0]

params = [ a => 1.0, b => 100.0]

#  Niether gradient nor Hessian are needed

prob = OptimizationProblem(sys, u0, params, grad=false, hess=false)

callback = function (p,l)
    println("Objective function value =  $l")
    return false
end

#  Define upper and lower limits for x and y.  First
#  element in the two vectors below corresponds to x and
#  the second element corresponds to y.

lower_limit = [-5, -5]

upper_limit = [5, 5]

#  Particle Swarm method may needs lots of iterations. 100 search
#  particles will be used here.

         result = GalacticOptim.solve(prob, 
                    ParticleSwarm(lower = lower_limit, upper = upper_limit, 
                   n_particles = 100), x_abstol = 1e-12, f_abstol = 1e-12, 
                      g_abstol = 1e-12, cb = callback, 
                      show_every = 10, time_limit = 200, iterations=100);

#  Note that, due to the constraints, the x=y=1 minimimum cannot be achieved.

println(" ")
println("Final value of objective function = ", result.minimum)
println(" ")
println("Final value for x = ", result.minimizer[1])
println(" ")
println("Final value for y = ", result.minimizer[2])

In [ ]:
#  Example problem 5.

#  Minimize the Rosenbrock function
#  with an equality constraint on x and y.

#  The problem solved is:

#       minimize:   (1 - x)^2 + 100 * (y - x^2)^2
#       subject to:     x*y = 2.0

#  Using the penalty function method, the above problem
#  can be solved as follows:

#    minimize (1-x)^2 + 100*(y-x^2)^2 + penalty_factor*(x*y-2.0)^2
#       where  penalty_factor is a large number (e.g., 10^6)

#  The BFGS method will be used. 

#  Define the variables and parameters.

@variables x y 
@parameters a b penalty_factor

#  Remainder of statements are similar to the first two examples
#  so most comments are eliminated below.

objective = (a - x)^2 + b * (y - x^2)^2  + penalty_factor*(x*y-2.0)^2

@named  sys = OptimizationSystem(objective, [x,y], [a,b,penalty_factor])

u0 = [ x => 1.0, y=> 1.0]

params = [ a => 1.0, b => 100.0, penalty_factor => 10^6]

prob = OptimizationProblem(sys,u0, params , grad=true, hess=false)

callback = function (p,l)
    println("Objective function value =  $l")
    return false
end

#  Broyden, Fletcher, Golfarb, Shanno (BFGS) method will be used

result = solve(prob, BFGS(),x_abstol = 1e-12, 
                   f_abstol = 1e-12, g_abstol = 1e-12, cb = callback, 
                    show_every = 5, time_limit = 100 , iterations=100)

#  Determine error in equality constraint

error1 = result.minimizer[1]*result.minimizer[2] - 2.0

println(" ")
println("Final value of objective function = ", result.minimum)
println(" ")
println("Final error in eq. constraint = ", error1)
println(" ")
println("Final value for x = ", result.minimizer[1])
println(" ")
println("Final value for y = ", result.minimizer[2])

In [ ]:
#  Example problem 6 

#  Minimize the Rosenbrock function with
#  an inequality constraint on x and y.

#  The problem solved is:

#       minimize:   (1 - x)^2 + 100 * (y - x^2)^2
#       subject to:     x^2 + y^2 is less than or equal to 0.5

#  Using the penalty function method, the above problem
#  can be solved as follows:

#    minimize (1-x)^2 + 100*(y-x^2)^2 + penalty_factor*max(0,(x^2+y^2.0 - 0.5))^2
#       where  penalty_factor is a large number (e.g., 10^6)

#  The BFGS method will be used. 

#  Define the variables and parameters.

@variables x y 
@parameters a b penalty_factor

#  Remainder of statements are similar to the first two examples
#  so most comments are eliminated below.

objective = (a - x)^2 + b * (y - x^2)^2  + 
                   penalty_factor*(max(0,x^2 + y^2 - 0.5))^2

@named  sys = OptimizationSystem(objective, [x,y], [a,b,penalty_factor])

u0 = [ x => 0.0, y=> 0.0]

params = [ a => 1.0, b => 100.0, penalty_factor => 10^6]

prob = OptimizationProblem(sys,u0, params , grad=true, hess=false)

callback = function (p,l)
    println("Objective function value =  $l")
    return false
end

#  Broyden, Fletcher, Golfarb, Shanno (BFGS) method will be used

result = solve(prob, BFGS(),x_abstol = 1e-12, 
                   f_abstol = 1e-12, g_abstol = 1e-12, cb = callback, 
                    show_every = 5, time_limit = 100 , iterations=100)

#  Determine error in equality constraint

x2_plus_y2 = result.minimizer[1]^2 + result.minimizer[2]^2  

println(" ")
println("Final value of objective function = ", result.minimum)
println(" ")
println("Final value of x2 + y2 = ", x2_plus_y2)
println(" ")
println("Final value for x = ", result.minimizer[1])
println(" ")
println("Final value for y = ", result.minimizer[2])

In [ ]:
#  Example problem 7 

#  The Rosenbrock function is minimized as in 
#  example problem 2 except vector notation 
#  will be used here.  To better 
#  illustrate the use of vectors, two independent
#  problems will be solved simultaneously using
#  a single objective function.  Specifically, 
#  the vectors w and par1 are used for the first 
#  problem where w[1] = x,  w[2] = y, par1[1] = 1.0 
#  and par1[2] = 100.0.  A similar correspondence applies 
#  to v and par2 for the second problem.

#  Define the variables and parameters. 

@variables w[1:2]  v[1:2]
@parameters par1[1:2] par2[1:2]

#  Define the objective function

objective = (par1[1] - w[1])^2 + par1[2] * (w[2] - w[1]^2)^2 +
               (par2[1] - v[1])^2 + par2[2] * (v[2] - v[1]^2)^2

#  Note that argument splatting (...) is used below

@named  sys = OptimizationSystem(objective, [w...; v...], [par1...; par2...])

#  Remainder of statements are similar to those above
#  so most comments are eliminated below.

w0 = [ w[1] => -1.0, w[2] => -2.0,  v[1] => -2.0, v[2] => -2.0]

params = [par1[1] => 1.0, par1[2] => 100.0, par2[1] => 2.0, par2[2] => 100.0]

prob = OptimizationProblem(sys, w0, params, grad=true, hess=false)

callback = function (p,l)
    println("Objective function value =  $l")
    return false
end

#  Broyden, Fletcher, Golfarb, Shanno (BFGS) method will be used

result = solve(prob, BFGS(),x_abstol = 1e-12, 
                   f_abstol = 1e-12, g_abstol = 1e-12, cb = callback, 
                    show_every = 5, time_limit = 100 , iterations=100)

println(" ")
println("Final value of objective function = ", result.minimum)
println(" ")
println("Final value for w[1] = ", result.minimizer[1])
println(" ")
println("Final value for w[2] = ", result.minimizer[2])
println(" ")
println("Final value for v[1] = ", result.minimizer[3])
println(" ")
println("Final value for v[2] = ", result.minimizer[4])

In [ ]:
#  Additional notes:

#  1.  If there are no  parameters in the problem,
#      then you can eliminate the @parameters statement. But
#      you still  need to include an empty vector (i.e., [])
#      elsewhere where the parameters are supposed to 
#      appear, such as in the following two statements:

#   @named  sys = OptimizationSystem(objective, [x,y], [])
#   prob = OptimizationProblem(sys,u0,[],grad=true,hess=false)

#  2.  If you want to solve a system of algebraic equations
#      such as:

#             f(x,y) = 0   and g(x,y) = 0

#      you can reformulate the above problem into the
#      following equivalent optimization problem:

#           minimize:    (f(x,y))^2 + (g(x,y))^2

#      The above optimzation problem can be solved as shown
#      by the examples above.

# 3.   For large problems the BFGS method may use a large amount
#      of memory.  In this case use the low-memory usage version of
#      this algorithm by replacing BFGS() with LBFGS().

# 4.   Note the following options for the function solve()
#             x_abstol = error in independent variables
#             f_abstol = error in the value of objective function
#             g_abstol = error in the gradient (must be zero at min)

# 5.   A summary of which optmization method to use for various
#      problems is as follows: 

#      For a low-dimensional problem with analytic gradients and Hessians, 
#      use the Newton method with trust region. For larger problems or 
#      when there is no analytic Hessian, use LBFGS. If the function is 
#      non-differentiable, use Nelder-Mead. Use these methods with 
#      multiple starting guesses if the problem is "non-convex" 
#      so that multiple local optima (i.e., multiple minima) exit.  More 
#      simply, use the Particle Swarm method with upper and lower bounds
#      for nonconvex problems.  (Adapted from:  
#       https://julianlsolvers.github.io/Optim.jl/stable/#user/algochoice/)

# 6.  Particle Swarm algorithm is described here:

#        https://en.wikipedia.org/wiki/Particle_swarm_optimization